In [4]:
import pandas as pd

In [5]:
campaign_valid_ids = pd.read_csv('done/sample_experiment/valid_samples_10000.csv')
target = pd.read_csv('done/target_whole.csv',index_col=0)
user = pd.read_csv('preprocessing/user_ppd_0508.csv',index_col = 0)

In [6]:
campaign_valid_ids = campaign_valid_ids['0'].to_list()

In [7]:
# valid filter 된 campaign_valid_ids 를 기준으로 user segment 와 target log filtering

sampled_user = user[user['userid'].isin(campaign_valid_ids)]
target = target[target['user'].isin(campaign_valid_ids)]

In [8]:
# # 코드 이후에 segment 뽑기 
# train_ = pd.read_csv('done/to_pang10_2/price.csv')
# test_ = pd.read_csv('done/to_pang10_2/price_t.csv')
# train_segment = sampled_user[sampled_user['user_id'].isin(train_.user_id.to_list())]
# test_segment = sampled_user[sampled_user['user_id'].isin(test_.user_id.to_list())]
# train_segment.to_csv('done/to_pang10_2/train_seg.csv')
# test_segment.to_csv('done/to_pang10_2/test_seg.csv')

In [9]:
print('valid IDs num :', len(campaign_valid_ids))
print('valid Campaign values num:', target.campaign_id.nunique())

valid IDs num : 115167
valid Campaign values num: 9990


User dataframe

In [10]:
sampled_user.rename(columns = {'userid' : 'user_id'}, inplace = True)

# Encode 
sampled_user['gender'] = sampled_user['gender'] - 1 
sampled_user['shopping_level'] = sampled_user['shopping_level']-1 

/var/folders/3m/c1zxtgn548xdfpfrwzglvk3c0000gn/T/ipykernel_5676/483463035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_user.rename(columns = {'userid' : 'user_id'}, inplace = True)
/var/folders/3m/c1zxtgn548xdfpfrwzglvk3c0000gn/T/ipykernel_5676/483463035.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_user['gender'] = sampled_user['gender'] - 1
/var/folders/3m/c1zxtgn548xdfpfrwzglvk3c0000gn/T/ipykernel_5676/483463035.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [11]:
user_str = sampled_user.copy()
user_str ['gender'] = sampled_user['gender'] + 100 
user_str ['age_level'] = sampled_user['age_level'] + 200 
user_str ['pvalue_level'] = sampled_user['pvalue_level'] + 300 
user_str ['shopping_level'] = sampled_user['shopping_level'] + 400 

user_str = user_str.applymap(lambda x: str(x))
sampled_user['segment'] = user_str['cms_group_id'] + ' ' + user_str['gender'] + ' ' + user_str['age_level'] + ' ' + user_str['pvalue_level'] + ' ' + user_str['shopping_level']


/var/folders/3m/c1zxtgn548xdfpfrwzglvk3c0000gn/T/ipykernel_5676/3858261494.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_user['segment'] = user_str['cms_group_id'] + ' ' + user_str['gender'] + ' ' + user_str['age_level'] + ' ' + user_str['pvalue_level'] + ' ' + user_str['shopping_level']


In [12]:
sampled_user.head()

,user_id,cms_group_id,gender,age_level,pvalue_level,shopping_level,segment
3,1670,4,1,4,0.0,0,4 101 204 300.0 400
4,2545,10,0,4,0.0,2,10 100 204 300.0 402
8,6355,1,1,1,1.0,2,1 101 201 301.0 402
26,12873,5,1,5,0.0,2,5 101 205 300.0 402
33,15455,2,1,2,2.0,2,2 101 202 302.0 402


## Train_test_spliting 
- Whole IDs count : 115167
- Train : 100000
- Test : 15167

In [13]:
len(campaign_valid_ids)

115167

In [14]:
import random 

# 1811개의 랜덤 요소를 test_ids 리스트에 넣기
test_ids = campaign_valid_ids[:15167]

# 나머지 20000개의 요소를 train_ids 리스트에 넣기
train_ids = campaign_valid_ids[15167:]

# 결과 출력
print(f"Test IDs : {len(test_ids)}")
print(f"Train IDs : {len(train_ids)}")

Test IDs : 15167
Train IDs : 100000


In [15]:
# id 를 기준으로 target 과 user 나누기 

train_user = sampled_user[sampled_user['user_id'].isin(train_ids)]
train_target = target[target['user'].isin(train_ids)]

test_user = sampled_user[sampled_user['user_id'].isin(test_ids)]
test_target = target[target['user'].isin(test_ids)]

## Making Sequence

In [13]:
# sequential 이 담길 DataFrame 생성
# 이하 sequence_df 에 넣기 

# cate_id = pd.DataFrame(columns=['user_id', 'num_user', 'seq_space_sep', 'label'])
# campaign_id = pd.DataFrame(columns=['user_id', 'num_user', 'seq_space_sep', 'label'])
# price = pd.DataFrame(columns=['user_id', 'num_user', 'seq_space_sep', 'label'])

# cate_t = pd.DataFrame(columns=['user_id', 'num_user', 'seq_space_sep', 'label'])
# campaign_t = pd.DataFrame(columns=['user_id', 'num_user', 'seq_space_sep', 'label'])
# price_t = pd.DataFrame(columns=['user_id', 'num_user', 'seq_space_sep', 'label'])

In [16]:
def sequence(target_df, target_column):
    prev_user = None
    num_user = 0
    seq = ''
    label = None

    target_ = target_df[['user', target_column, 'from_purchase']]
    sequence_df = pd.DataFrame(columns=['user_id', 'num_user', 'seq_space_sep', 'label'])

    for i, j in target_.iterrows():
        current_user = j['user']
        
        # 현재 사용자와 이전 사용자가 다를 경우
        if current_user != prev_user:
            if label == 0 and prev_user is not None:
                sequence_df = pd.concat([sequence_df, pd.DataFrame([{'user_id': prev_user, 'num_user': num_user, 'seq_space_sep': seq, 'label': label}])], ignore_index=True)
            # 초기화 및 user 재설정 
            prev_user = current_user
            seq = ''
            num_user = 0
            
        # from_purchase가 1이면 label을 1로 설정하고 test_seq에 추가
        if j['from_purchase'] == 1:
            label = j['from_purchase']
            sequence_df = pd.concat([sequence_df, pd.DataFrame([{'user_id': current_user, 'num_user': num_user, 'seq_space_sep': seq, 'label': label}])], ignore_index=True)
            seq = ''
            num_user += 1
        else:
            label = j['from_purchase']
            seq += str(j[target_column]) + ' '
        
    # 마지막 사용자에 대한 데이터 추가
    if prev_user is not None:
        sequence_df = pd.concat([sequence_df, pd.DataFrame([{'user_id': current_user, 'num_user': num_user, 'seq_space_sep': seq, 'label': label}])], ignore_index=True)
    
    return sequence_df


## CATE ID 

In [19]:
# train 
cate_id = sequence(train_target,'cate_id')

# test
cate_t = sequence(test_target,'cate_id')

# CAMPAIGN ID 

In [24]:
# train 
campaign_id = sequence(train_target,'campaign_id')

# test
campaign_t = sequence(test_target,'campaign_id')

# PRICE SEQ

In [25]:
# train 
price = sequence(train_target,'price')

# test
price_t = sequence(test_target,'price')

check & SAVE

In [33]:
cate_id =cate_id[cate_id['seq_space_sep'] != ""]
cate_t = cate_t[cate_t['seq_space_sep'] != ""]
campaign_id =campaign_id[campaign_id['seq_space_sep'] != ""]
campaign_t = campaign_t[campaign_t['seq_space_sep'] != ""]
price = price[price['seq_space_sep'] != ""]
price_t = price_t[price_t['seq_space_sep'] != ""]

In [34]:
print('train shape: ',cate_id.shape,campaign_id.shape,price.shape)
print('test shape: ',cate_t.shape,campaign_t.shape,price_t.shape)

train shape:  (110710, 4) (110710, 4) (110710, 4)
test shape:  (16819, 4) (16819, 4) (16819, 4)


In [38]:
cate_id.to_csv('done/to_pang10_3/cate_train.csv')
cate_t.to_csv('done/to_pang10_3/cate_test.csv')
campaign_id.to_csv('done/to_pang10_3/camp_train.csv')
campaign_t.to_csv('done/to_pang10_3/camp_test.csv')
price.to_csv('done/to_pang10_3/price_train.csv')
price_t.to_csv('done/to_pang10_3/price_test.csv')

In [36]:
# 코드 이후에 segment 뽑기 
train_user = cate_id.user_id.to_list()
test_user = cate_t.user_id.to_list()


train_segment = sampled_user[sampled_user['user_id'].isin(train_user)]
test_segment = sampled_user[sampled_user['user_id'].isin(test_user)]
train_segment.to_csv('done/to_pang10_3/train_seg.csv')
test_segment.to_csv('done/to_pang10_3/test_seg.csv')

In [40]:
price.label.value_counts(normalize=True)

0.0    0.568323
1.0    0.431677
Name: label, dtype: float64